<a href="https://colab.research.google.com/github/alexgshaw/vol3project2/blob/main/The_Generated_Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Grant White, Johnson Merrell, Walker Hughes, Alex Shaw

# Introduction

Can we train a computer to write new church hymns? Our project is seeking to answer this question. We will be attempting to recreate both lyrics and music. In this draft we focus on text generation. 

Text generation has become a hot topic in the past decade, and has begun seeing great success in recent years. Applying these methods to song generation will be a new aspect to explore. Music generation has not seen the same amount of success as text generation, but is still an active field of research.

# Data Cleaning

We will use the lyrics from all the Hymns of the Church of Jesus Christ of Latter-day Saints as well as the instrumental tracks. Both of these are available at www.churchofjesuschrist.org. There are 341 hymns available in English.

The first step will be processing our data. This will be done in the following five steps:

1. Split each word into it's own line.
1. Remove verse numbers.
1. Replace sentence-ending punctuation and end of songs with stop words.
1. Remove all remaining punctuation.

The code for data cleaning is given below.